In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait2(s):
    pass
wait = wait1

def call(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv

### Global functions and routines

In [3]:
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def settemp(sp):
    if sp == 0:
        call(h.settemp, 2, 37)
    else:
        call(h.settemp, 0, sp)
        
def endbatch():
    call(h.endbatch)
    
def pid1():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 12)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 29)
    
def pid2():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 13)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26)
    
def pid3():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 14)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26)

In [4]:
_bni = 1
def nb(pref, i=1, n=1):
    bl = call(h.getBatches)
    b = bl.names_to_batches
    s = "%s %d.%d" % (pref, i, n)
    while s in b:
        n += 1
        s = "%s %d.%d" % (pref, i, n)
    startbatch(s)

def tempoff():
    settemp(0)
    
def setbn(i):
    global _bni
    _bni = i

def o2a():
    nb("tpid", _bni)
    settemp(37)
    wait(3*hours)
    endbatch()
    
def cool(hrs=8):
    tempoff()
    wait(hrs*hours)
    
def a2a(end):
    nb("tpid", _bni)
    settemp(end)
    wait(3*hours)
    endbatch()

def verify(pid):
    print("Verifying settings")
    pid()
    o2a()
    a2a(35)
    a2a(37)
    
def pid4():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 13)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 28) 

## Round 4 5/24/17
Temp looked a bit unstable in real life in the bio lab. According to simulation, a much higher gain value doens't actually hurt performance, so trying the test out again. 

In [5]:
def pid5():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 50)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26) 
hours = 3600

In [11]:
h = HelloApp('71.189.82.196:81')
wait = wait1
cool(10)
setbn(1)
verify(pid5)
tempoff()

Calling settemp(2, 37)
Sleeping 36000 seconds, ending at 05/25/2017 03:16:34 AM
Verifying settings
Calling setconfig('Temperature', 'P_Gain_(%/C)', 50)
Calling setconfig('Temperature', 'I_Time_(min)', 26)
Calling getBatches()
Calling batchrunning()
Calling startbatch('tpid 1.1')
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 05/25/2017 06:16:40 AM
Calling endbatch()
Calling getBatches()
Calling batchrunning()
Calling startbatch('tpid 1.2')
Calling settemp(0, 35)
Sleeping 10800 seconds, ending at 05/25/2017 09:16:43 AM
Calling endbatch()
Calling getBatches()
Calling batchrunning()
Calling startbatch('tpid 1.3')
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 05/25/2017 12:16:46 PM
Calling endbatch()
Calling settemp(2, 37)


In [16]:
h = HelloApp('71.189.82.196:81')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\PID Tuning\\Temperature\\Raw Data\\'

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+"3L " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [19]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
    
def analyze(fn):
    c1 = cells.Find("TempPV(C)")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "1440"
    mkch(c1,"TempPV(C)")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
#     for v in "TempHeatDutyActual(%)",:
#         c = cells.Find(v).Offset(2,1)
#         mk_elapsed(c, ref)
#         mkch(c,v)
#         mkser(ch2, c, v)
    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
        
def analf(f):
    open_file(f)
    analyze(f)

In [21]:
bl = ['tpid 1.%d' % i for i in (1,2,3)]
files = []
for b in bl:
    fn = dl(b)
    files.append(fn)
xl = Excel()
for file in files: analf(file)

Calling getdatareport_bybatchname('tpid 1.1')
Calling getdatareport_bybatchname('tpid 1.2')
Calling getdatareport_bybatchname('tpid 1.3')
